In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
cd /content/drive/MyDrive/exercise

/content/drive/MyDrive/exercise


In [17]:
import os
import re

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity


from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import pickle

import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import warnings
warnings.filterwarnings('ignore')

In [7]:
## adult 전처리 완료 저장
data = pd.read_csv('./cleaned_data/adult_2019_1.csv')

In [8]:
data.shape

(49433, 18)

In [9]:
data.columns

Index(['회원일련번호값', '측정연령수', '측정일자', '성별구분코드', '인증구분명', '신장(cm)', '체중(kg)',
       '체지방율(%)', '앉아윗몸앞으로굽히기(cm)', 'BMI(kg/㎡)', '교차윗몸일으키기(회)', '왕복오래달리기(회)',
       '10M 4회 왕복달리기(초)', '제자리 멀리뛰기(cm)', '상대악력(%)', '준비운동', '본운동', '마무리운동'],
      dtype='object')

In [10]:
data.head()

,회원일련번호값,측정연령수,측정일자,성별구분코드,인증구분명,신장(cm),체중(kg),체지방율(%),앉아윗몸앞으로굽히기(cm),BMI(kg/㎡),교차윗몸일으키기(회),왕복오래달리기(회),10M 4회 왕복달리기(초),제자리 멀리뛰기(cm),상대악력(%),준비운동,본운동,마무리운동
0,AAG5scSkx90/QuAl06tA8wuC,59,20190102,F,참가증,159.0,57.5,32.0,15.5,22.7,1.0,11.0,15.76,126.0,41.0,"상지 루틴 스트레칭,전신 루틴 스트레칭","윗몸올리기,엎드려 버티기,누워서 다리 들어올리기,누워서 수직으로 다리 들어올리기,누...","하지 루틴 스트레칭,하지 루틴 스트레칭,전신 루틴 스트레칭"
1,AAHMUHU8ZnWT2Dqdgq5QIMrm,60,20190102,F,참가증,160.3,59.4,30.7,21.2,23.1,0.0,8.0,16.03,107.0,30.1,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
2,AAEIkFejDxJhMG7x6KPtE45Z,64,20190102,M,2등급,163.9,59.3,24.2,14.1,22.1,30.0,20.0,13.00,169.0,61.5,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
3,AAGOaVPSy8aq6s1PTwzEsgfx,42,20190102,M,참가증,167.6,73.1,26.8,19.6,26.0,32.0,20.0,11.63,191.0,64.5,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
4,AAECwHF8wknRv812NQT09fxu,32,20190102,F,1등급,167.9,56.4,19.3,22.0,20.0,35.0,35.0,11.94,192.0,50.0,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."


# 유저 그룹 정보 분리해 저장

In [ ]:
data = pd.get_dummies(data = data, columns = ['성별구분코드'], prefix = '성별구분코드')

In [ ]:
info_col = ["회원일련번호값","측정연령수","측정일자"]
category_col = ["성별구분코드_F", "성별구분코드_M"]
body_col = ["신장(cm)","체중(kg)","체지방율(%)"]
# 신장, 체중, 신체 구성, 
health_col = ["앉아윗몸앞으로굽히기(cm)","BMI(kg/㎡)","교차윗몸일으키기(회)","왕복오래달리기(회)","10M 4회 왕복달리기(초)","제자리 멀리뛰기(cm)","상대악력(%)"]
# 유연성, 체질량지수, 근지구력, 심폐지구력, 민첩성, 순발력, 근력
ex_col = ["준비운동","본운동","마무리운동"]
target = ["인증구분명"]

In [ ]:

adult_group_0 = data[data["인증구분명"]=="참가증"][category_col+body_col+health_col+ex_col]
adult_group_0["index"] = list(range(0, len(adult_group_0)))
adult_group_0=adult_group_0.set_index("index")


adult_group_0.to_csv('./cleaned_data/adult_group/adult_2019_0.csv',encoding='utf-8',index=False)

In [ ]:
adult_group_1 = data[data["인증구분명"]=="3등급"][category_col+body_col+health_col+ex_col]

adult_group_1["index"] = list(range(0, len(adult_group_1)))
adult_group_1=adult_group_1.set_index("index")


adult_group_1.to_csv('./cleaned_data/adult_group/adult_2019_1.csv',encoding='utf-8',index=False)

In [ ]:
adult_group_2 = data[data["인증구분명"]=="2등급"][category_col+body_col+health_col+ex_col]

adult_group_2["index"] = list(range(0, len(adult_group_2)))
adult_group_2=adult_group_2.set_index("index")


adult_group_2.to_csv('./cleaned_data/adult_group/adult_2019_2.csv',encoding='utf-8',index=False)

In [ ]:
adult_group_3 = data[data["인증구분명"]=="1등급"][category_col+body_col+health_col+ex_col]

adult_group_3["index"] = list(range(0, len(adult_group_3)))
adult_group_3=adult_group_3.set_index("index")


adult_group_3.to_csv('./cleaned_data/adult_group/adult_2019_3.csv',encoding='utf-8',index=False)

# 그룹별 데이터 읽어오기

In [11]:
## adult group 0
adult_group_0 = pd.read_csv('./cleaned_data/adult_group/adult_2019_0.csv')
## adult group 1
adult_group_1 = pd.read_csv('./cleaned_data/adult_group/adult_2019_1.csv')
## adult group 0
adult_group_2 = pd.read_csv('./cleaned_data/adult_group/adult_2019_2.csv')
## adult group 0
adult_group_3 = pd.read_csv('./cleaned_data/adult_group/adult_2019_3.csv')

In [12]:
adult_group_0.head(7)

,성별구분코드_F,성별구분코드_M,신장(cm),체중(kg),체지방율(%),앉아윗몸앞으로굽히기(cm),BMI(kg/㎡),교차윗몸일으키기(회),왕복오래달리기(회),10M 4회 왕복달리기(초),제자리 멀리뛰기(cm),상대악력(%),준비운동,본운동,마무리운동
0,1,0,159.0,57.5,32.0,15.5,22.7,1.0,11.000000,15.76,126.0,41.0,"상지 루틴 스트레칭,전신 루틴 스트레칭","윗몸올리기,엎드려 버티기,누워서 다리 들어올리기,누워서 수직으로 다리 들어올리기,누...","하지 루틴 스트레칭,하지 루틴 스트레칭,전신 루틴 스트레칭"
1,1,0,160.3,59.4,30.7,21.2,23.1,0.0,8.000000,16.03,107.0,30.1,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
2,0,1,167.6,73.1,26.8,19.6,26.0,32.0,20.000000,11.63,191.0,64.5,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
3,0,1,167.1,87.8,29.6,8.2,31.4,44.0,28.000000,11.59,189.0,48.1,"깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,어깨 뒤쪽 스트레칭,아래...","몸통 들어올리기,소파를 이용한 윗몸올리기,엎드려 버티기,옆으로 누워 버티기,네발기기...","깍지 끼고 상체 숙이기,목 스트레칭,등/어깨 뒤쪽 스트레칭,아래 팔 스트레칭,넙다리..."
4,0,1,171.4,82.1,25.0,-2.0,27.9,39.0,30.510531,11.30,196.0,52.6,걷기,"달리기,무릎 높여 제자리 달리기",전신 루틴 스트레칭
5,1,0,164.7,60.4,32.3,2.1,22.3,20.0,12.000000,15.01,127.0,38.4,"목 스트레칭,등/어깨 뒤쪽 스트레칭,가슴/어깨 앞쪽 스트레칭,아래 팔 스트레칭,하지...","의자 앞에서 앉았다 일어서기,엎드려 버티기,누워서 엉덩이 들어올리기,네발기기 자세로...","걷기,계단 올라갔다 내려오기,수영,옆구리 스트레칭,넙다리 스트레칭,넙다리 스트레칭"
6,0,1,166.3,63.7,20.2,1.1,23.0,48.0,47.000000,10.15,242.0,73.9,"하지 루틴 스트레칭,상지 루틴 스트레칭",전신 루틴 스트레칭,자가근막이완술 루틴 스트레칭


In [13]:
# 유저가 0 그룹인 경우 - 기존의 0 그룹 테이블 가져오기 (0 ~ 3)
adult_group_0.shape

(29280, 15)

In [14]:
adult_group_0.tail()

,성별구분코드_F,성별구분코드_M,신장(cm),체중(kg),체지방율(%),앉아윗몸앞으로굽히기(cm),BMI(kg/㎡),교차윗몸일으키기(회),왕복오래달리기(회),10M 4회 왕복달리기(초),제자리 멀리뛰기(cm),상대악력(%),준비운동,본운동,마무리운동
29275,0,1,175.3,78.10,26.0,24.1,25.4,46.0,42.000000,10.00,188.0,59.6,"걷기,유산소 운동 전 동적 루틴 스트레칭","맨몸운동 루틴프로그램,팔벌려뛰기,사다리 옆으로 발 옮기기,옆으로 펀치하기,스텝퍼 오...",전신 루틴 스트레칭
29276,0,1,165.0,68.66,20.4,-3.7,25.2,20.0,30.510531,13.80,172.0,56.9,걷기,"달리기,무릎 높여 제자리 달리기",전신 루틴 스트레칭
29277,1,0,168.5,63.40,35.4,1.7,22.3,28.0,16.000000,12.36,146.0,42.1,"걷기,줄넘기,전신 루틴 스트레칭,유산소 운동 전 동적 루틴 스트레칭","서서 상체 일으키기,앉았다 일어서기,소파를 이용한 윗몸올리기,엎드려 버티기,네발기기...","하지 루틴 스트레칭,하지 루틴 스트레칭,상지 루틴 스트레칭,자가근막이완술 루틴 스트레칭"
29278,0,1,180.5,96.30,24.2,4.6,29.6,29.0,30.510531,12.06,184.0,44.8,"상지 루틴 스트레칭,하지 루틴 스트레칭,하지 루틴 스트레칭,전신 루틴 스트레칭,유산...","탄력밴드 어깨 가쪽 돌림,서서 탄력밴드 당겨내리기/올리기,엎드려 버티기,누워서 다리...","목 스트레칭,허리 스트레칭,배 스트레칭,짐볼을 이용한 동적 루틴 스트레칭,작은 공을..."
29279,0,1,173.0,83.60,24.1,15.9,27.9,39.0,52.000000,10.66,211.0,54.4,"전신 루틴 스트레칭,유산소 운동 전 동적 루틴 스트레칭","윗몸 말아 올리기,윗몸올리기,누워서 다리 들어올리기,누워서 팔 다리 동시에 들어올리...",전신 루틴 스트레칭


# 유사 유저 구하기

In [ ]:
# 새로운 유저 정보 - 가짜 데이터
new_user_dict = {'성별구분코드_F' : 1, '성별구분코드_M' : 0, '신장(cm)' : 159, 
                            "체중(kg)":57.5, "체지방율(%)":32.00, "앉아윗몸앞으로굽히기(cm)":15.5, 
                            "BMI(kg/㎡)":22.70, "교차윗몸일으키기(회)" : 1, "왕복오래달리기(회)":11.00,
                            "10M 4회 왕복달리기(초)":15.76, "제자리 멀리뛰기(cm)":126, "상대악력(%)":41.00
                            }

In [42]:
# 마지막에 추가하는 거 함수화
def add_NewUser(group, new_user):
  # 체력평가등급 0 그룹 내 piv 테이블 생성에 필요한 칼러만 가져오기
  category_col = ["성별구분코드_F", "성별구분코드_M"]
  body_col = ["신장(cm)","체중(kg)","체지방율(%)"]
  # 신장, 체중, 신체 구성, 
  health_col = ["앉아윗몸앞으로굽히기(cm)","BMI(kg/㎡)","교차윗몸일으키기(회)","왕복오래달리기(회)","10M 4회 왕복달리기(초)","제자리 멀리뛰기(cm)","상대악력(%)"]
  group_table = group[category_col+body_col + health_col]
  # 새로운 유저 정보 + 기존 체력 평가 등급 0 그룹 테이블
  total_user = group_table.append(new_user, ignore_index=True)
  return total_user


In [33]:
def create_CF(total_user):
  CF = pd.DataFrame(cosine_similarity(total_user))
  CF.index = total_user.index
  CF.columns = total_user.index
  return CF


In [35]:
def getCF(userId,CF, ex_type, top_n):
    Id = userId
    idx = CF[Id].sort_values(ascending = False)[1:top_n].index.values
    viewed_list = data[data.index==Id][ex_type].values
    ex_list = []
    for i in idx:
        other_viewed_list = data[data.index==i][ex_type].values
        for j in other_viewed_list:
            if j not in viewed_list:
              ex_list.append(j.split(","))
    
    #print('====={} 운동 추천 목록 ====='.format(ex_type))
    #for i in ex_list:
      #print(i)
    
    return ex_list

In [ ]:
# 가장 비슷한 상위 10% 유저의 준비 운동 목록 

# 새로운 유저 포함 테이블 생성
total_user = add_NewUser(adult_group_0, new_user_dict)
# 새로운 유저 번호 부여
new_user_idx = len(total_user)-1
# 새로운 유저와 유사한 상위 10% 유저의 운동 처방 가져오기 - 새로운 유저, 본운동
top_10percent_ex_list = getCF(new_user_idx, create_CF(total_user), "본운동", int(data.shape[0]*(10/100)))
print("new user idx", len(total_user))
#print(top_10percent_ex_list)

new user idx 29281
[['달리기', '자전거타기', '줄넘기', '계단 올라갔다 내려오기', '계단 뛰기', '수영', '팔벌려뛰기', '버피운동', '목 스트레칭', '등/어깨 뒤쪽 스트레칭', '허리 스트레칭', '엉덩이 스트레칭', '넙다리 스트레칭', '넙다리 스트레칭', '넙다리 스트레칭', '하지 루틴 스트레칭', '하지 루틴 스트레칭', '상지 루틴 스트레칭', '전신 루틴 스트레칭'], ['앉았다 일어서기', '한발 앞으로 내밀고 앉았다 일어서기', '넙다리 스트레칭', '하지 루틴 스트레칭', '상지 루틴 스트레칭', '좌식생활자를 위한 동적 루틴 스트레칭 '], ['윗몸올리기', '누워서 엉덩이 들어올리기', '엎드려 버티기', '서서 상체 일으키기', '앉아서 밀기', '앉아서 당겨 내리기', '앉아서 다리 밀기', '앉아서 다리 펴기', '자전거타기', '걷기'], ['상지 루틴 스트레칭', '전신 루틴 스트레칭'], ['팔굽혀펴기', '허리 굽혀 팔 뒤로 들기', '앉았다 일어서기', '달리기', '계단 올라갔다 내려오기'], ['달리기', '무릎 높여 제자리 달리기'], ['앉았다 일어서기', '한발 앞으로 내밀고 앉았다 일어서기', '턱걸이', '점프 팔굽혀펴기', '소파를 이용한 윗몸올리기', '엎드려 버티기'], ['달리기', '자전거타기', '줄넘기', '계단 올라갔다 내려오기'], ['달리기', '자전거타기', '줄넘기'], ['달리기', '자전거타기', '자전거타기', '줄넘기', '계단 올라갔다 내려오기'], ['앉았다 일어서기', '팔굽혀펴기', '윗몸올리기', '누워서 엉덩이 들어올리기', '네발기기 자세로 팔 다리 들기', '엎드려 버티기', '누워서 수직으로 다리 들어올리기', '누워서 다리 좌우로 움직이기'], ['몸통 들어올리기', '소파를 이용한 윗몸올리기', '엎드려 버티기', '옆으로 누워 버티기', '네발기기 자세로 팔 다리 들기', '엎드려서 팔 다리 들기'], ['맨몸운동 루틴프로그램']

# APRIOR 

In [47]:
def get_sparse_matrix(df):
  te = TransactionEncoder()
  te_ary = te.fit(df).transform(df)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_) #위에서 나온걸 보기 좋게 데이터프레임으로 변경
  return sparse_matrix

In [48]:
def get_freq_item(sparse_matrix, min_support): # 0.01
  #print("시작")
  frequent_itemsets = apriori(sparse_matrix, min_support=min_support, use_colnames=True)
  #print("완료")
  return frequent_itemsets

In [49]:
def get_association_rules(frequent_itemsets, min_threshold): #0.8
  association_rules_df = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_threshold)
  return association_rules_df

In [50]:
def get_apriori_result(result,min_support, min_confidence):
  sparse_matrix = get_sparse_matrix(result)
  frequent_itemsets = get_freq_item(sparse_matrix, min_support)
  association_rules_df = get_association_rules(frequent_itemsets,min_confidence)
  return association_rules_df

In [51]:
# consequents 1개인 거 -> 지지도, 신뢰도, lift 내림차순
def get_sorted(ar_df):
  ar_df_sorted = ar_df[ar_df["consequents"].apply(lambda x : len(x) == 1)].sort_values(by=["support","confidence","lift"], ascending=False)
  ar_df_sorted["consequents"] = ar_df_sorted["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode").apply(lambda x: x.split(','))
  ar_df_sorted["antecedents"] = ar_df_sorted["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode").apply(lambda x: x.split(','))
  return ar_df_sorted

In [52]:
def create_top5_ex(ar_df_sorted):
  total = []
  for idx in ar_df_sorted.index:
    temp = ar_df_sorted.loc[idx,'consequents']+ar_df_sorted.loc[idx,'antecedents'] 
    for ex  in temp:
      if ex not in total:
        total.append(ex)
  length = 5 if len(total) else len(total) 
  return total[:length]

In [53]:
def get_top5_ex(ar_df):
  ar_df_sorted = get_sorted(ar_df)
  ex_rec_list = create_top5_ex(ar_df_sorted)
  return ex_rec_list

In [ ]:
ar_df = get_apriori_result(top_10percent_ex_list,min_support = 0.1, min_confidence=0.8)

완
시작
완료
완
완


In [ ]:
get_top5_ex(ar_df)

['자전거타기', '걷기', '수영', '앉았다 일어서기', '한발 앞으로 내밀고 앉았다 일어서기']

# 테스트

## 그룹별 데이터 pickle 저장

In [22]:
# 그룹 0 데이터 저장
with open('./cleaned_data/pickle/adult_group/adult_group_0.pickle', 'wb') as f:
    pickle.dump(adult_group_0, f)

In [23]:
# 그룹 1 데이터 저장
with open('./cleaned_data/pickle/adult_group/adult_group_1.pickle', 'wb') as f:
    pickle.dump(adult_group_1, f)

In [24]:
# 그룹 2 데이터 저장
with open('./cleaned_data/pickle/adult_group/adult_group_2.pickle', 'wb') as f:
    pickle.dump(adult_group_2, f)

In [25]:
# 그룹 3 데이터 저장
with open('./cleaned_data/pickle/adult_group/adult_group_3.pickle', 'wb') as f:
    pickle.dump(adult_group_3, f)

## 그룹별 데이터 불러오기

In [27]:
# load
with open('./cleaned_data/pickle/adult_group/adult_group_0.pickle', 'rb') as f:
    adult_group_0_pickle = pickle.load(f)

# 유사 유저 함수 저장

In [44]:
# 모델 저장
with open('./model/similiar_user/add_NewUser.pickle','wb') as fw:
    pickle.dump(add_NewUser, fw)

In [34]:
# 모델 저장
with open('./model/similiar_user/create_CF.pickle','wb') as fw:
    pickle.dump(create_CF, fw)

In [36]:
with open('./model/similiar_user/getCF.pickle','wb') as fw:
    pickle.dump(getCF, fw)

## 유사 유저 함수 불러오기

In [45]:
# load
with open('./model/similiar_user/add_NewUser.pickle', 'rb') as f:
    add_NewUser1 = pickle.load(f)

In [38]:
# load
with open('./model/similiar_user/create_CF.pickle', 'rb') as f:
    create_CF1 = pickle.load(f)

In [39]:
# load
with open('./model/similiar_user/getCF.pickle', 'rb') as f:
    getCF1 = pickle.load(f)

## 유사 유저 구하기 test

In [40]:
# 새로운 유저 정보 - 가짜 데이터
new_user_dict = {'성별구분코드_F' : 1, '성별구분코드_M' : 0, '신장(cm)' : 159, 
                            "체중(kg)":57.5, "체지방율(%)":32.00, "앉아윗몸앞으로굽히기(cm)":15.5, 
                            "BMI(kg/㎡)":22.70, "교차윗몸일으키기(회)" : 1, "왕복오래달리기(회)":11.00,
                            "10M 4회 왕복달리기(초)":15.76, "제자리 멀리뛰기(cm)":126, "상대악력(%)":41.00
                            }

In [46]:
total_user = add_NewUser1(adult_group_0, new_user_dict)
# 새로운 유저 번호 부여
new_user_idx = len(total_user)-1
# 새로운 유저와 유사한 상위 10% 유저의 운동 처방 가져오기 - 새로운 유저, 본운동
top_10percent_ex_list = getCF1(new_user_idx, create_CF1(total_user), "본운동", int(data.shape[0]*(10/100)))
print("new user idx", len(total_user))
#print(top_10percent_ex_list)

new user idx 29281


# 운동 추천 함수 저장

In [55]:
# 모델 저장
with open('./model/rec_nonPreference/get_sparse_matrix.pickle','wb') as fw:
    pickle.dump(get_sparse_matrix, fw)

In [56]:
# get_freq_item
with open('./model/rec_nonPreference/get_freq_item.pickle','wb') as fw:
    pickle.dump(get_freq_item, fw)

In [57]:
# get_association_rules
with open('./model/rec_nonPreference/get_association_rules.pickle','wb') as fw:
    pickle.dump(get_association_rules, fw)

In [58]:
# get_apriori_result
with open('./model/rec_nonPreference/get_apriori_result.pickle','wb') as fw:
    pickle.dump(get_apriori_result, fw)

In [59]:
# get_sorted
with open('./model/rec_nonPreference/get_sorted.pickle','wb') as fw:
    pickle.dump(get_sorted, fw)

In [60]:
# create_top5_ex
with open('./model/rec_nonPreference/create_top5_ex.pickle','wb') as fw:
    pickle.dump(create_top5_ex, fw)

In [61]:
# get_top5_ex
with open('./model/rec_nonPreference/get_top5_ex.pickle','wb') as fw:
    pickle.dump(get_top5_ex, fw)

## 운동 추천 함수 불러오기

In [62]:
# load
with open('./model/rec_nonPreference/get_sparse_matrix.pickle', 'rb') as f:
    get_sparse_matrix = pickle.load(f)

In [65]:
# load
with open('./model/rec_nonPreference/get_freq_item.pickle', 'rb') as f:
    get_freq_item = pickle.load(f)

In [66]:
# load
with open('./model/rec_nonPreference/get_association_rules.pickle', 'rb') as f:
    get_association_rules = pickle.load(f)

In [67]:
# load
with open('./model/rec_nonPreference/get_apriori_result.pickle', 'rb') as f:
    get_apriori_result = pickle.load(f)

In [68]:
# load
with open('./model/rec_nonPreference/get_sorted.pickle', 'rb') as f:
    get_sorted = pickle.load(f)

In [69]:
# load
with open('./model/rec_nonPreference/create_top5_ex.pickle', 'rb') as f:
    create_top5_ex = pickle.load(f)

In [70]:
# load
with open('./model/rec_nonPreference/get_top5_ex.pickle', 'rb') as f:
    get_top5_ex = pickle.load(f)

## 운동 추천 test

In [71]:
ar_df = get_apriori_result(top_10percent_ex_list,min_support = 0.1, min_confidence=0.8)

In [72]:
get_top5_ex(ar_df)

['자전거타기', '걷기', '수영', '앉았다 일어서기', '한발 앞으로 내밀고 앉았다 일어서기']